<a href="https://colab.research.google.com/github/Harinadhavasala/Chatbot_Code/blob/main/Copy_of_Data_lake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [322]:
import pandas as pd

In [323]:

df = pd.read_excel("Casa_Grand_Lead_Sample_Data.xlsx")

In [324]:
df.head()

,LastName,FirstName,Salutation,Name,Company,Phone,MobilePhone,Email,SYS_Mobile__c,WA_Formatted_Phone__c,...,Mobile8__c,Mobile9__c,Mobile10__c,Country_Code__c,First_Phone__c,Second_Phone__c,Third_Phone__c,Fourth_Phone__c,Sixth_Phone__c,Seventh_Phone__c
0,As Chakravarthi,NaN,NaN,As Chakravarthi,As Chakravarthi,919962154351,NaN,aschakrav@gmail.com,nul,9.199622e+11,...,NaN,NaN,NaN,91,9962154351,9.080857e+09,8.778092e+09,NaN,NaN,NaN
1,Raveendra,Meda,NaN,Meda Raveendra,Meda Raveendra,919962041716,9.199620e+11,mrreddy@ramcadds.in,919962041716,9.199620e+11,...,NaN,NaN,NaN,91,9962041716,NaN,NaN,NaN,NaN,NaN
2,Kannan,NaN,Mr,Kannan,Kannan,918189923399,9.181899e+11,kannan.ican1891@gmail.com,918189923399,9.181899e+11,...,NaN,NaN,NaN,91,8189923399,NaN,NaN,NaN,NaN,NaN
3,Ramyaa,NaN,Mr,Ramyaa,Ramyaa,919962746446,9.199627e+11,ramyaa.doc@gmail.com,919962746446,9.199627e+11,...,NaN,NaN,NaN,91,9962746446,9.003088e+09,NaN,NaN,NaN,NaN
4,Vasan.K,NaN,Mr,Vasan.K,Vasan.K,918939149522,9.189391e+11,vasankm@yahoo.com,918939149522,9.189391e+11,...,NaN,NaN,NaN,91,8939149522,NaN,NaN,NaN,NaN,NaN


In [325]:
text_cols = df.select_dtypes(include="object").columns
df[text_cols] = df[text_cols].fillna("")

In [326]:
import re
for col in ['FirstName', 'LastName']:
    df[col] = df[col].astype(str)
    df[col] = df[col].str.replace(r'[^a-zA-Z. ]', '', regex=True)



In [327]:
df.head(15)

,LastName,FirstName,Salutation,Name,Company,Phone,MobilePhone,Email,SYS_Mobile__c,WA_Formatted_Phone__c,...,Mobile8__c,Mobile9__c,Mobile10__c,Country_Code__c,First_Phone__c,Second_Phone__c,Third_Phone__c,Fourth_Phone__c,Sixth_Phone__c,Seventh_Phone__c
0,As Chakravarthi,,,As Chakravarthi,As Chakravarthi,919962154351,NaN,aschakrav@gmail.com,nul,9.199622e+11,...,NaN,NaN,NaN,91,9962154351,9.080857e+09,8.778092e+09,NaN,NaN,NaN
1,Raveendra,Meda,,Meda Raveendra,Meda Raveendra,919962041716,9.199620e+11,mrreddy@ramcadds.in,919962041716,9.199620e+11,...,NaN,NaN,NaN,91,9962041716,NaN,NaN,NaN,NaN,NaN
2,Kannan,,Mr,Kannan,Kannan,918189923399,9.181899e+11,kannan.ican1891@gmail.com,918189923399,9.181899e+11,...,NaN,NaN,NaN,91,8189923399,NaN,NaN,NaN,NaN,NaN
3,Ramyaa,,Mr,Ramyaa,Ramyaa,919962746446,9.199627e+11,ramyaa.doc@gmail.com,919962746446,9.199627e+11,...,NaN,NaN,NaN,91,9962746446,9.003088e+09,NaN,NaN,NaN,NaN
4,Vasan.K,,Mr,Vasan.K,Vasan.K,918939149522,9.189391e+11,vasankm@yahoo.com,918939149522,9.189391e+11,...,NaN,NaN,NaN,91,8939149522,NaN,NaN,NaN,NaN,NaN
5,sundar,,Mr,sundar,sundar,919962502711,9.199625e+11,ksundar2001@gmail.com,919962502711,9.199625e+11,...,NaN,NaN,NaN,91,9962502711,NaN,NaN,NaN,NaN,NaN
6,Pandiyan,,Mr,Pandiyan,Pandiyan,919790947384,9.197909e+11,acp3012@gmail.com,919790947384,9.197909e+11,...,NaN,NaN,NaN,91,9790947384,NaN,NaN,NaN,NaN,NaN
7,K.Uma,,Mr,K.Uma,K.Uma,919840891528,9.198409e+11,kuma189@yahoo.com,919840891528,9.198409e+11,...,NaN,NaN,NaN,91,9840891528,NaN,NaN,NaN,NaN,NaN
8,Vinodh Kumar,,,Vinodh Kumar,Vinodh Kumar,919940194192,9.199402e+11,vinoths99401@gmail.com,919940194192,9.199402e+11,...,NaN,NaN,NaN,91,9940194192,NaN,NaN,NaN,NaN,NaN
9,Soundar,,,Soundar,Soundar,919677023477,9.196770e+11,phyraj2008@gmail.com,919677023477,9.196770e+11,...,NaN,NaN,NaN,91,9677023477,NaN,NaN,NaN,NaN,NaN


In [328]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("NameValidation") \
    .getOrCreate()


In [329]:
spark_df = spark.createDataFrame(df)


In [330]:
from pyspark.sql import functions as F

spark_df = spark_df.withColumn(
    "full_name",
    F.trim(F.concat_ws(" ", F.col("FirstName"), F.col("LastName")))
)


In [331]:
spark_df.show(truncate=False)



+---------------+---------------+----------+---------------------------+---------------------------+------------+----------------+-------------------------------+-------------+---------------------+-------------------------------+--------------------+----------------+----------------+---------------+----------+----------+----------+----------+----------+----------+-----------+---------------+--------------+---------------+--------------+---------------+--------------+----------------+---------------------------+
|LastName       |FirstName      |Salutation|Name                       |Company                    |Phone       |MobilePhone     |Email                          |SYS_Mobile__c|WA_Formatted_Phone__c|Email__c                       |Encrypted_SMobile__c|Mobile_Phone__c |Phone__c        |Mobile3__c     |Mobile4__c|Mobile5__c|Mobile6__c|Mobile7__c|Mobile8__c|Mobile9__c|Mobile10__c|Country_Code__c|First_Phone__c|Second_Phone__c|Third_Phone__c|Fourth_Phone__c|Sixth_Phone__c|Seventh_Ph

In [332]:
# from pyspark.sql import functions as F

# spark_df = spark_df.withColumn(
#     "invalid_category",

#     # 1. Empty or blank
#     F.when(
#         F.col("full_name").rlike(r"^\s*$"),
#         "CATEGORY_EMPTY_NAME"
#     )

#     # 2. Blocking / placeholder words
#     .when(
#         F.col("full_name").rlike(
#             r"(?i)\b(do not call|dont call|no call|remove|xyz|abcd|abc|test|poly|elangovan)\b"
#         ),
#         "Invalid Name"
#     )

#     # 3. Only single-letter / dot initial
#     .when(
#         F.col("full_name").rlike(r"^\s*\.?[A-Za-z]\.?\s*$"),
#         "CATEGORY_ONLY_INITIAL"
#     )
# )


In [333]:
from pyspark.sql import functions as F

spark_df = spark_df.withColumn(
    "invalid_names",

    # 1. Empty or blank
    F.when(
        F.col("full_name").rlike(r"^\s*$"),
        "Category_empty_name"
    )

    # 2. Only dots (., .., ...)
    .when(
        F.col("full_name").rlike(r"^\s*\.{1,}\s*$"),
        "Category_only_dots"
    )

    # 3. Only single initial (R, R., .R)
    .when(
        F.col("full_name").rlike(r"^\s*\.?[A-Za-z]\.?\s*$"),
        "Category_Only_Initial"
    )

    # 4. Blocking / placeholder words
    .when(
        F.col("full_name").rlike(
            r"(?i)\b(do not call|dont call|no call|remove|xyz|abcd|abc|test|poly|elangovan)\b"
        ),
        "  Category_Invalid_Name"
    )
)


In [334]:
import pyspark.sql.functions as F

spark_df = spark_df.withColumn(
    "is_valid_names",
    F.when(F.col("invalid_names").isNotNull(), False)  # Invalid names → False
     .otherwise(None)                                 # Valid names → blank
)


In [335]:
spark_df.show(10,truncate=False)


+---------------+---------+----------+---------------+---------------+------------+----------------+-------------------------+-------------+---------------------+-------------------------+--------------------+----------------+----------------+---------------+----------+----------+----------+----------+----------+----------+-----------+---------------+--------------+---------------+--------------+---------------+--------------+----------------+---------------+-------------+--------------+
|LastName       |FirstName|Salutation|Name           |Company        |Phone       |MobilePhone     |Email                    |SYS_Mobile__c|WA_Formatted_Phone__c|Email__c                 |Encrypted_SMobile__c|Mobile_Phone__c |Phone__c        |Mobile3__c     |Mobile4__c|Mobile5__c|Mobile6__c|Mobile7__c|Mobile8__c|Mobile9__c|Mobile10__c|Country_Code__c|First_Phone__c|Second_Phone__c|Third_Phone__c|Fourth_Phone__c|Sixth_Phone__c|Seventh_Phone__c|full_name      |invalid_names|is_valid_names|
+-------------

In [336]:
from pyspark.sql import functions as F

spark_df = (
    spark_df
    # convert FirstName and LastName to lowercase
    .withColumn("FirstName", F.lower(F.col("FirstName")))
    .withColumn("LastName", F.lower(F.col("LastName")))

    # drop full_name column
    .drop("full_name")
)


In [337]:
from pyspark.sql.functions import col, trim, when

EMAIL_REGEX = (
    r'^(?=.{1,254}$)(?=.{1,64}@)'
    r'[A-Za-z0-9_%+-]+(\.[A-Za-z0-9_%+-]+)*'
    r'@'
    r'[A-Za-z0-9-]+(\.[A-Za-z0-9-]+)*\.[A-Za-z]{2,}$'
)

spark_df = spark_df.withColumn(
    "is_valid_email",
    when(col("Email").isNull(), False)  # Missing email → False
    .when(~trim(col("Email")).rlike(EMAIL_REGEX), False)  # Invalid email → False
    .otherwise(None)  # Valid email → blank/NULL
)


In [338]:
spark_df.show(truncate=False)


+---------------+---------------+----------+---------------------------+---------------------------+------------+----------------+-------------------------------+-------------+---------------------+-------------------------------+--------------------+----------------+----------------+---------------+----------+----------+----------+----------+----------+----------+-----------+---------------+--------------+---------------+--------------+---------------+--------------+----------------+-------------+--------------+--------------+
|LastName       |FirstName      |Salutation|Name                       |Company                    |Phone       |MobilePhone     |Email                          |SYS_Mobile__c|WA_Formatted_Phone__c|Email__c                       |Encrypted_SMobile__c|Mobile_Phone__c |Phone__c        |Mobile3__c     |Mobile4__c|Mobile5__c|Mobile6__c|Mobile7__c|Mobile8__c|Mobile9__c|Mobile10__c|Country_Code__c|First_Phone__c|Second_Phone__c|Third_Phone__c|Fourth_Phone__c|Sixth_Pho

In [339]:
# import pyspark.sql.functions as F

# valid_email_df = spark_df.filter(F.col("is_valid_email") == True)

# valid_email_df.show(truncate=False)


In [340]:
# import pyspark.sql.functions as F
# from pyspark.sql.window import Window

# # Window to count occurrences of each email
# email_window = Window.partitionBy("Email")

# spark_df = spark_df.withColumn(
#     "email_status",
#     F.when(
#         F.col("is_valid_email") == False,
#         "Invalid"
#     )
#     .when(
#         (F.col("is_valid_email") == True) &
#         (F.count("Email").over(email_window) > 1),
#         "Duplicate"
#     )
#     .otherwise("Unique")
# )


In [341]:
import pyspark.sql.functions as F
from pyspark.sql.window import Window

# Preserve original row order
spark_df = spark_df.withColumn("original_order", F.monotonically_increasing_id())

# Window for duplicate detection
email_window = Window.partitionBy("Email")

spark_df = spark_df.withColumn(
    "email_status",
    F.when(
        F.col("Email").isNull() |
        (F.trim(F.col("Email")) == "") |
        (F.lower(F.trim(F.col("Email"))) == "nan"),
        "Email Is Not Present"
    )
    .when(F.col("is_valid_email") == False, "Invalid Email")
    .when(F.count("Email").over(email_window) > 1, "Duplicate")
    .otherwise(None)   # Valid & unique → blank
)

# Restore original order
spark_df = spark_df.orderBy("original_order").drop("original_order")


In [342]:
spark_df.show(10,truncate=False)


+---------------+---------+----------+---------------+---------------+------------+----------------+-------------------------+-------------+---------------------+-------------------------+--------------------+----------------+----------------+---------------+----------+----------+----------+----------+----------+----------+-----------+---------------+--------------+---------------+--------------+---------------+--------------+----------------+-------------+--------------+--------------+------------+
|LastName       |FirstName|Salutation|Name           |Company        |Phone       |MobilePhone     |Email                    |SYS_Mobile__c|WA_Formatted_Phone__c|Email__c                 |Encrypted_SMobile__c|Mobile_Phone__c |Phone__c        |Mobile3__c     |Mobile4__c|Mobile5__c|Mobile6__c|Mobile7__c|Mobile8__c|Mobile9__c|Mobile10__c|Country_Code__c|First_Phone__c|Second_Phone__c|Third_Phone__c|Fourth_Phone__c|Sixth_Phone__c|Seventh_Phone__c|invalid_names|is_valid_names|is_valid_email|emai

In [343]:
df = spark_df.toPandas()


In [344]:
df.to_csv("output.csv", index=False)


In [321]:
df.to_csv(r"C:\Users\Vasala\Desktop\datasets\lead_data.csv", index=False)
